In [1]:
import pandas as pd 
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import requests
import plotly.express as px
import plotly.graph_objects as go


## 1 - OUVERTURE FICHIER EQUIPEMENT CULTUREL


In [69]:
df_lieu_equip_cult = pd.read_csv('./Data/base-des-lieux-et-des-equipements-culturels.csv', sep =";")
df_lieu_equip_cult.head(5)

/tmp/ipykernel_316368/4195843643.py:1: DtypeWarning: Columns (14,22,23,29,30,33,34,41,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lieu_equip_cult = pd.read_csv('./Data/base-des-lieux-et-des-equipements-culturels.csv', sep =";")


,Source,Nom,Adresse,Complement Adresse,Code Postal,libelle_geographique,code_insee,Code Insee Arrondt,Identifiant origine,Type équipement ou lieu,...,Annee_Label_Appellation,Latitude,Longitude,Demographie_AP,Demographie_detail_entree,Démographie_detail_sortie,Multi_equipements,identifiant_QPV_24,identifiant_QPV_15,coordonnees_geo
0,Ministère de la Culture - Service du livre et ...,Bibliothèque de GAMARde,348 RUE ABBE BORDES,NaN,40380.0,Gamarde-les-Bains,40104,401,LI7887,Bibliothèque,...,NaN,43.7325530845461,-0.873206140912455,Actif,historiquement dans la base en 2020,NaN,NaN,NaN,NaN,"43.7325530845461, -0.873206140912455"
1,Ministère de la Culture - Service du livre et ...,Médiathèque de GRENAde-SUR-L'ADOUR,32 RUE DES CAPUCINS,NaN,40270.0,Grenade-sur-l'Adour,40117,402,LI7890,Bibliothèque,...,NaN,43.772152205146,-0.429404631079996,Actif,historiquement dans la base en 2020,NaN,NaN,NaN,NaN,"43.772152205146, -0.429404631079996"
2,Ministère de la Culture - Service du livre et ...,LUDO-Bibliothèque MUNICIPALE de LABENNE,14 RUE DU PRESBYTERE,NaN,40530.0,Labenne,40133,401,LI1735,Bibliothèque,...,NaN,43.5959632032077,-1.42775664273915,Actif,historiquement dans la base en 2020,NaN,NaN,NaN,NaN,"43.5959632032077, -1.42775664273915"
3,Ministère de la Culture - Service du livre et ...,Bibliothèque MUNICIPALE de LABOUHEYRE,46 RUE FRANÇOIS MITTERRAND,NaN,40210.0,Labouheyre,40134,402,LI1736,Bibliothèque,...,NaN,44.2133421363569,-0.918764424591646,Actif,historiquement dans la base en 2020,NaN,NaN,NaN,NaN,"44.2133421363569, -0.918764424591646"
4,Ministère de la Culture - Service du livre et ...,Médiathèque de LABRIT,30 ROUTE DE LUXEY,NaN,40420.0,Labrit,40135,402,LI7899,Bibliothèque,...,NaN,44.1050991899747,-0.545001121981277,Actif,historiquement dans la base en 2020,NaN,NaN,NaN,NaN,"44.1050991899747, -0.545001121981277"


In [70]:
df_lieu_equip_cult.iloc[:,10].unique()


array([nan, 'Centre d’art contemporain d’intérêt national',
       'Centre chorégraphique national',
       'Centre de développement chorégaphique national',
       'Centre dramatique national', 'Art et essai',
       'Architecture contemporaine remarquable',
       'Centre national des arts de la rue et de l’espace public',
       "Fonds régional d'art contemporain", 'Jardin remarquable',
       'LIR - Librairie indépendante de référence',
       'LR - Librairie de référence', 'Maison des illustres',
       'Microfolie', 'Monument national', 'Monument historique',
       'Musée de France', 'Opéra national en région',
       'Pôle national cirque', 'Centre culturel de rencontre',
       'Scène conventionnée d’intérêt national', 'Scène nationale',
       'Scène de musiques actuelles', 'Théâtre hors label',
       'Théâtre national', 'Théâtre privé', 'Théâtre de ville',
       "Patrimoine mondial de l'Unesco", 'Zénith',
       'Site patrimonial remarquable',
       'Centre national de la

In [71]:
df_lieu_equip_cult.to_csv('df_lieu_equip_cult.csv', index=False)

# 2- NEW FICHIER POPULATION



In [72]:
df_population= pd.read_csv('./Data/datapopulation.csv', sep =";", header = 2)
df_population

,Code,Libellé,Population municipale 2021
0,01001,L'Abergement-Clémenciat,832
1,01002,L'Abergement-de-Varey,267
2,01004,Ambérieu-en-Bugey,14854
3,01005,Ambérieux-en-Dombes,1897
4,01006,Ambléon,113
...,...,...,...
34930,97613,M'Tsangamouji,N/A - résultat non disponible
34931,97614,Ouangani,N/A - résultat non disponible
34932,97615,Pamandzi,N/A - résultat non disponible
34933,97616,Sada,N/A - résultat non disponible


In [73]:
df_population.isna().sum()

Code                          0
Libellé                       0
Population municipale 2021    0
dtype: int64

In [74]:
df_population['Population municipale 2021'].value_counts()['N/A - résultat non disponible']

20

In [75]:
len(df_population['Population municipale 2021'])

34935

In [76]:
df_population['Population municipale 2021'] = df_population['Population municipale 2021'].map(lambda x : 0 if x == 'N/A - résultat non disponible' else x)

In [77]:
'N/A - résultat non disponible'

'N/A - résultat non disponible'

In [78]:
df_population

,Code,Libellé,Population municipale 2021
0,01001,L'Abergement-Clémenciat,832
1,01002,L'Abergement-de-Varey,267
2,01004,Ambérieu-en-Bugey,14854
3,01005,Ambérieux-en-Dombes,1897
4,01006,Ambléon,113
...,...,...,...
34930,97613,M'Tsangamouji,0
34931,97614,Ouangani,0
34932,97615,Pamandzi,0
34933,97616,Sada,0


In [79]:
df_population['Population municipale 2021']

0          832
1          267
2        14854
3         1897
4          113
         ...  
34930        0
34931        0
34932        0
34933        0
34934        0
Name: Population municipale 2021, Length: 34935, dtype: object

In [80]:
df_population.to_csv('df_population.csv', index=False)


2/  - POPULATION Création nouveau datframe pour villes fdont population supérieur > 20 000 habitants et < 20 000 habitants

In [81]:
df_population

,Code,Libellé,Population municipale 2021
0,01001,L'Abergement-Clémenciat,832
1,01002,L'Abergement-de-Varey,267
2,01004,Ambérieu-en-Bugey,14854
3,01005,Ambérieux-en-Dombes,1897
4,01006,Ambléon,113
...,...,...,...
34930,97613,M'Tsangamouji,0
34931,97614,Ouangani,0
34932,97615,Pamandzi,0
34933,97616,Sada,0


In [82]:
df_population_integrer = df_population['Population municipale 2021'].astype(int)
df_population_integrer

0          832
1          267
2        14854
3         1897
4          113
         ...  
34930        0
34931        0
34932        0
34933        0
34934        0
Name: Population municipale 2021, Length: 34935, dtype: int64

In [83]:
df_grandes_villes = df_population_integrer[df_population_integrer > 20000]
df_grandes_villes

47        41525
239       22277
781       24021
1053      52958
1081      28705
          ...  
34908    105240
34909     84077
34911     34344
34913     24293
34915     81943
Name: Population municipale 2021, Length: 481, dtype: int64

In [84]:
df_petites_villes = df_population_integrer[df_population_integrer < 20000]
df_petites_villes


0          832
1          267
2        14854
3         1897
4          113
         ...  
34930        0
34931        0
34932        0
34933        0
34934        0
Name: Population municipale 2021, Length: 34454, dtype: int64

In [85]:
len(df_petites_villes) 

34454

## 3/ GEOPOINTS -ouverture fichier

In [86]:

df_geopoints= pd.read_csv('./Data/villes_france_etl.csv', sep =",")


df_geopoints.head()

,insee_code,city_code,zip_code,label,latitude,longitude,department_name,department_number,region_name,region_geojson_name
0,25620,ville du pont,25650,ville du pont,46.999873,6.498147,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
1,25624,villers grelot,25640,villers grelot,47.361512,6.235167,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
2,25615,villars les blamont,25310,villars les blamont,47.368384,6.871415,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
3,25619,les villedieu,25240,les villedieu,46.713906,6.265831,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
4,25622,villers buzon,25170,villers buzon,47.228558,5.852187,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté


3/ GEOPOINTS - Récupérer Latitude et Longitude avec codes postaux 

In [87]:
df_geopoints.rename(columns={'zip_code': 'Codepostaux'}, inplace=True)

In [88]:
df_geopoints

,insee_code,city_code,Codepostaux,label,latitude,longitude,department_name,department_number,region_name,region_geojson_name
0,25620,ville du pont,25650,ville du pont,46.999873,6.498147,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
1,25624,villers grelot,25640,villers grelot,47.361512,6.235167,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
2,25615,villars les blamont,25310,villars les blamont,47.368384,6.871415,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
3,25619,les villedieu,25240,les villedieu,46.713906,6.265831,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
4,25622,villers buzon,25170,villers buzon,47.228558,5.852187,doubs,25,bourgogne-franche-comté,Bourgogne-Franche-Comté
...,...,...,...,...,...,...,...,...,...,...
39140,98829,thio,98829,thio,NaN,NaN,nouvelle-calédonie,988,nouvelle-calédonie,Nouvelle Calédonie
39141,98831,voh,98833,voh,NaN,NaN,nouvelle-calédonie,988,nouvelle-calédonie,Nouvelle Calédonie
39142,98832,yate,98834,yate,NaN,NaN,nouvelle-calédonie,988,nouvelle-calédonie,Nouvelle Calédonie
39143,98612,sigave,98620,sigave,-14.270411,-178.155263,wallis-et-futuna,986,wallis-et-futuna,Wallis-et-Futuna


In [89]:
df_geopoints = df_geopoints[['Codepostaux','latitude','longitude']]
df_geopoints

,Codepostaux,latitude,longitude
0,25650,46.999873,6.498147
1,25640,47.361512,6.235167
2,25310,47.368384,6.871415
3,25240,46.713906,6.265831
4,25170,47.228558,5.852187
...,...,...,...
39140,98829,NaN,NaN
39141,98833,NaN,NaN
39142,98834,NaN,NaN
39143,98620,-14.270411,-178.155263


In [90]:
df_geopoints.dropna()

,Codepostaux,latitude,longitude
0,25650,46.999873,6.498147
1,25640,47.361512,6.235167
2,25310,47.368384,6.871415
3,25240,46.713906,6.265831
4,25170,47.228558,5.852187
...,...,...,...
38929,97650,-12.751168,45.086393
38930,97670,-12.837567,45.138321
39104,98799,10.297389,-109.221264
39143,98620,-14.270411,-178.155263


In [91]:
df_geopoints['Codepostaux'] = df_geopoints['Codepostaux'].astype(str)

/tmp/ipykernel_316368/1988093705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geopoints['Codepostaux'] = df_geopoints['Codepostaux'].astype(str)


In [92]:
df_geopoints.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39145 entries, 0 to 39144
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Codepostaux  39145 non-null  object 
 1   latitude     38934 non-null  float64
 2   longitude    38934 non-null  float64
dtypes: float64(2), object(1)
memory usage: 917.6+ KB


2/ GEOPOINTS  - Création d'une nouvelle DataFrame Concatenee pour geopoint et population 

In [93]:
concatenated_dataframe = pd.concat([df_geopoints, df_population], axis=1) 
concatenated_dataframe.head()

,Codepostaux,latitude,longitude,Code,Libellé,Population municipale 2021
0,25650,46.999873,6.498147,01001,L'Abergement-Clémenciat,832
1,25640,47.361512,6.235167,01002,L'Abergement-de-Varey,267
2,25310,47.368384,6.871415,01004,Ambérieu-en-Bugey,14854
3,25240,46.713906,6.265831,01005,Ambérieux-en-Dombes,1897
4,25170,47.228558,5.852187,01006,Ambléon,113


In [94]:
concatenated_dataframe['zip'] = list(zip(concatenated_dataframe['latitude'], concatenated_dataframe['longitude']))

concatenated_dataframe['zip'] 

#print(concatenated_dataframe[['latitude', 'longitude', 'zip']].head())


0            (46.999873398, 6.498147193)
1            (47.361512085, 6.235167025)
2            (47.368383721, 6.871414913)
3             (46.713906258, 6.26583065)
4            (47.228558434, 5.852186748)
                      ...               
39140                         (nan, nan)
39141                         (nan, nan)
39142                         (nan, nan)
39143    (-14.270411199, -178.155263035)
39144    (-13.281862538, -176.161927918)
Name: zip, Length: 39145, dtype: object

In [95]:
concatenated_dataframe['zip'] .isna().sum()

0

In [96]:
concatenated_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39145 entries, 0 to 39144
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Codepostaux                 39145 non-null  object 
 1   latitude                    38934 non-null  float64
 2   longitude                   38934 non-null  float64
 3   Code                        34935 non-null  object 
 4   Libellé                     34935 non-null  object 
 5   Population municipale 2021  34935 non-null  object 
 6   zip                         39145 non-null  object 
dtypes: float64(2), object(5)
memory usage: 2.1+ MB


In [97]:
concatenated_dataframe.dropna(inplace=True)
concatenated_dataframe.tail(5)

,Codepostaux,latitude,longitude,Code,Libellé,Population municipale 2021,zip
34930,80110,49.776139,2.484102,97613,M'Tsangamouji,0,"(49.776139288, 2.484101678)"
34931,80290,49.778731,1.815822,97614,Ouangani,0,"(49.778731195, 1.815822336)"
34932,80140,49.927364,1.665057,97615,Pamandzi,0,"(49.927364114, 1.665057388)"
34933,80110,49.743036,2.522593,97616,Sada,0,"(49.743035523, 2.52259271)"
34934,80240,50.004214,3.015558,97617,Tsingoni,0,"(50.004214454, 3.015558336)"


In [98]:
concatenated_dataframe.isnull().sum()

Codepostaux                   0
latitude                      0
longitude                     0
Code                          0
Libellé                       0
Population municipale 2021    0
zip                           0
dtype: int64

## 4/ NEW DATAFRAME MUSEE

In [99]:
df_musee = pd.read_csv('./Data/musees-de-france-base-museofile.csv', sep =";")
df_musee = pd.DataFrame(df_musee)
df_musee.head(5)

,Identifiant,Nom_officiel,Adresse,Lieu,Code_postal,Ville,Région,Departement,URL,Telephone,...,Thèmes,Artiste,Personnage_phare,Intérêt,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees
0,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Arts décoratifs : Céramique,NaN,NaN,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409"
1,M0018,musée des Arts Décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Arts décoratifs : Céramique, Mobilier, Horloge...","Robert de Cotte, Paul et Joseph Hannong (faïen...",Cardinaux Rohan,"Palais Rohan, construit de 1739 à 1742 par Rob...",NaN,Site patrimonial remarquable. Patrimoine mondi...,SPR4400040,NaN,2024-02-07,"48.581229, 7.751984"
2,M0019,musée des Beaux-Arts,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-beaux-arts,03 68 98 51 60,...,"Beaux-Arts : Peinture, Sculpture","Giotto, Botticelli, Raphaël, Véronèse, le Gue...",NaN,"Palais Rohan, construit de 1739 à 1742 par Rob...",Protégé au titre des Monuments historiques : h...,Site patrimonial remarquable. Patrimoine mondi...,PA00085184;SPR4400040,1801,2024-02-07,"48.581229, 7.751984"
3,M0025,musée gallo-romain,place de la mairie,le Capitole,68600,Biesheim,Grand Est,Haut-Rhin,www.biesheim.fr/culture-loisirs/musee-gallo-ro...,03 89 72 01 58,...,Archéologie nationale : Gallo-romain,NaN,NaN,Bâtiment réalisé en 1991 par le cabinet Mongie...,NaN,NaN,NaN,NaN,2024-02-07,"48.041667, 7.54802"
4,M0057,musée d'art et d'archéologie du Périgord,22 cours Tourny,musée,24000,Périgueux,Nouvelle-Aquitaine,Dordogne,www.perigueux-maap.fr/,05 53 06 40 70,...,"Antiquités étrangères : Grecques et romaines, ...","Oudry, Canaletto, Bouguereau, Pierre Huys","Wulgrinn de Taillefer, De Mourcin, Docteur Gal...",Le bâtiment actuel a été réalisé suite à un co...,Protégé au titre des Monuments historiques : h...,Site patrimonial remarquable.,PA24000099;SPR7500071;SPR7500072,NaN,2024-01-25,"45.186039, 0.72395"


In [100]:
df_musee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226 entries, 0 to 1225
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Identifiant          1226 non-null   object
 1   Nom_officiel         1226 non-null   object
 2   Adresse              1197 non-null   object
 3   Lieu                 482 non-null    object
 4   Code_postal          1226 non-null   object
 5   Ville                1226 non-null   object
 6   Région               1226 non-null   object
 7   Departement          1226 non-null   object
 8   URL                  1140 non-null   object
 9   Telephone            1200 non-null   object
 10  Categorie            330 non-null    object
 11  Domaine_thematique   1181 non-null   object
 12  Histoire             1171 non-null   object
 13  Atout                1161 non-null   object
 14  Thèmes               1080 non-null   object
 15  Artiste              506 non-null    object
 16  Person

In [101]:
df_musee.columns

Index(['Identifiant', 'Nom_officiel', 'Adresse', 'Lieu', 'Code_postal',
       'Ville', 'Région', 'Departement', 'URL', 'Telephone', 'Categorie',
       'Domaine_thematique', 'Histoire', 'Atout', 'Thèmes', 'Artiste',
       'Personnage_phare', 'Intérêt', 'Protection_batiment',
       'Protection_espace', 'Refmer', 'Annee_creation', 'Date_de_mise_a_jour',
       'Coordonnees'],
      dtype='object')

In [102]:
df_musee.columns = ['Identifiant', 'Nom_officiel', 'Adresse', 'Lieu', 'Codepostaux',
       'Ville', 'Région', 'Departement', 'URL', 'Telephone', 'Categorie',
       'Domaine_thematique', 'Histoire', 'Atout', 'Thèmes', 'Artiste',
       'Personnage_phare', 'Intérêt', 'Protection_batiment',
       'Protection_espace', 'Refmer', 'Annee_creation', 'Date_de_mise_a_jour',
       'Coordonnees']
df_musee

,Identifiant,Nom_officiel,Adresse,Lieu,Codepostaux,Ville,Région,Departement,URL,Telephone,...,Thèmes,Artiste,Personnage_phare,Intérêt,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees
0,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Arts décoratifs : Céramique,NaN,NaN,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409"
1,M0018,musée des Arts Décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Arts décoratifs : Céramique, Mobilier, Horloge...","Robert de Cotte, Paul et Joseph Hannong (faïen...",Cardinaux Rohan,"Palais Rohan, construit de 1739 à 1742 par Rob...",NaN,Site patrimonial remarquable. Patrimoine mondi...,SPR4400040,NaN,2024-02-07,"48.581229, 7.751984"
2,M0019,musée des Beaux-Arts,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-beaux-arts,03 68 98 51 60,...,"Beaux-Arts : Peinture, Sculpture","Giotto, Botticelli, Raphaël, Véronèse, le Gue...",NaN,"Palais Rohan, construit de 1739 à 1742 par Rob...",Protégé au titre des Monuments historiques : h...,Site patrimonial remarquable. Patrimoine mondi...,PA00085184;SPR4400040,1801,2024-02-07,"48.581229, 7.751984"
3,M0025,musée gallo-romain,place de la mairie,le Capitole,68600,Biesheim,Grand Est,Haut-Rhin,www.biesheim.fr/culture-loisirs/musee-gallo-ro...,03 89 72 01 58,...,Archéologie nationale : Gallo-romain,NaN,NaN,Bâtiment réalisé en 1991 par le cabinet Mongie...,NaN,NaN,NaN,NaN,2024-02-07,"48.041667, 7.54802"
4,M0057,musée d'art et d'archéologie du Périgord,22 cours Tourny,musée,24000,Périgueux,Nouvelle-Aquitaine,Dordogne,www.perigueux-maap.fr/,05 53 06 40 70,...,"Antiquités étrangères : Grecques et romaines, ...","Oudry, Canaletto, Bouguereau, Pierre Huys","Wulgrinn de Taillefer, De Mourcin, Docteur Gal...",Le bâtiment actuel a été réalisé suite à un co...,Protégé au titre des Monuments historiques : h...,Site patrimonial remarquable.,PA24000099;SPR7500071;SPR7500072,NaN,2024-01-25,"45.186039, 0.72395"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,M1037,musée des arts décoratifs,34 rue de la charité,NaN,69002,Lyon,Auvergne-Rhône-Alpes,Rhône,www.museedestissus.fr/,NaN,...,NaN,NaN,NaN,Les travaux de restauration de l’hôtel de Lacr...,Protégé au titre des Monuments historiques : h...,Site patrimonial remarquable. Site inscrit. Pa...,PA00117818;SPR8400131,NaN,2024-02-09,"45.75342759382487, 4.8312127576713175"
1222,M1222,musée de la Création Franche,58 avenue du Maréchal de Lattre-de-Tassigny,NaN,33130,Bègles,Nouvelle-Aquitaine,Gironde,NaN,05 56 85 81 73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-19,"44.80598303387724, -0.5434896909627212"
1223,M0542,musée Pierre Noël - musée de la vie dans les H...,place Georges Trimouille,NaN,88100,Saint-Dié-des-Vosges,Grand Est,Vosges,www.ca-saintdie.fr/decouvrir/musee-pierre-noel...,03 29 51 60 35,...,"Archéologie nationale : Préhistoire, Gallo-rom...","Jean-Baptiste Augustin, Gallé, Le Corbusier, L...","Jules Ferry, né à Saint-Dié. René Fonck, héros...",Ancien palais épiscopal (XVIIIe siècle) et édi...,Protégé au titre des Monuments historiques : h...,Site patrimonial remarquable. Abords d'un Monu...,PA00107282;ACR0000622;SPR4400051,NaN,2024-08-14,"48.28874228462725, 6.950459939493648"
1224,M1192,musée Vert Véron-de-Forbonnais,204 avenue Jean Jaurès,NaN,72100,Le Mans,Pays-de-la-Loire,Sarthe,www.lemans.fr/dynamique/des-idees-de-visite/le...,02 43 47 39 94,...,NaN,NaN,Louis-Marin Bachelier (1785-1861) ; Jean Bouvi...,Ancienne école construite à la fin du XIXe siè...,NaN,NaN,NaN,NaN,2023-04-12,"47.986868, 0.208417"


4/  MUSEE - add new colonne = Musée

In [103]:
df_musee['Musée'] = df_musee.apply(lambda _: 'Musée', axis=1)

In [104]:
df_musee.head(5)

,Identifiant,Nom_officiel,Adresse,Lieu,Codepostaux,Ville,Région,Departement,URL,Telephone,...,Artiste,Personnage_phare,Intérêt,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees,Musée
0,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,NaN,NaN,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée
1,M0018,musée des Arts Décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Robert de Cotte, Paul et Joseph Hannong (faïen...",Cardinaux Rohan,"Palais Rohan, construit de 1739 à 1742 par Rob...",NaN,Site patrimonial remarquable. Patrimoine mondi...,SPR4400040,NaN,2024-02-07,"48.581229, 7.751984",Musée
2,M0019,musée des Beaux-Arts,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-beaux-arts,03 68 98 51 60,...,"Giotto, Botticelli, Raphaël, Véronèse, le Gue...",NaN,"Palais Rohan, construit de 1739 à 1742 par Rob...",Protégé au titre des Monuments historiques : h...,Site patrimonial remarquable. Patrimoine mondi...,PA00085184;SPR4400040,1801,2024-02-07,"48.581229, 7.751984",Musée
3,M0025,musée gallo-romain,place de la mairie,le Capitole,68600,Biesheim,Grand Est,Haut-Rhin,www.biesheim.fr/culture-loisirs/musee-gallo-ro...,03 89 72 01 58,...,NaN,NaN,Bâtiment réalisé en 1991 par le cabinet Mongie...,NaN,NaN,NaN,NaN,2024-02-07,"48.041667, 7.54802",Musée
4,M0057,musée d'art et d'archéologie du Périgord,22 cours Tourny,musée,24000,Périgueux,Nouvelle-Aquitaine,Dordogne,www.perigueux-maap.fr/,05 53 06 40 70,...,"Oudry, Canaletto, Bouguereau, Pierre Huys","Wulgrinn de Taillefer, De Mourcin, Docteur Gal...",Le bâtiment actuel a été réalisé suite à un co...,Protégé au titre des Monuments historiques : h...,Site patrimonial remarquable.,PA24000099;SPR7500071;SPR7500072,NaN,2024-01-25,"45.186039, 0.72395",Musée


In [105]:
df_musee.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1226 entries, 0 to 1225
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Identifiant          1226 non-null   object
 1   Nom_officiel         1226 non-null   object
 2   Adresse              1197 non-null   object
 3   Lieu                 482 non-null    object
 4   Codepostaux          1226 non-null   object
 5   Ville                1226 non-null   object
 6   Région               1226 non-null   object
 7   Departement          1226 non-null   object
 8   URL                  1140 non-null   object
 9   Telephone            1200 non-null   object
 10  Categorie            330 non-null    object
 11  Domaine_thematique   1181 non-null   object
 12  Histoire             1171 non-null   object
 13  Atout                1161 non-null   object
 14  Thèmes               1080 non-null   object
 15  Artiste              506 non-null    object
 16  Person

    3/ MUSEE - Création Dataframe Concatenés Musée et géopoint

In [106]:
concat_musee_dataframe = pd.merge(df_musee, df_geopoints, on='Codepostaux')
concat_musee_dataframe

,Identifiant,Nom_officiel,Adresse,Lieu,Codepostaux,Ville,Région,Departement,URL,Telephone,...,Intérêt,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees,Musée,latitude,longitude
0,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
1,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
2,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
3,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
4,M0018,musée des Arts Décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Palais Rohan, construit de 1739 à 1742 par Rob...",NaN,Site patrimonial remarquable. Patrimoine mondi...,SPR4400040,NaN,2024-02-07,"48.581229, 7.751984",Musée,48.571265,7.767760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,M1218,"atelier-musée de l'imprimerie, Le Malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,NaN,NaN,NaN,NaN,NaN,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7381,M1218,"atelier-musée de l'imprimerie, Le Malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,NaN,NaN,NaN,NaN,NaN,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.253272,2.418897
7382,M1218,"atelier-musée de l'imprimerie, Le Malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,NaN,NaN,NaN,NaN,NaN,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7383,M1218,"atelier-musée de l'imprimerie, Le Malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,NaN,NaN,NaN,NaN,NaN,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945


In [107]:
concat_musee_dataframe

,Identifiant,Nom_officiel,Adresse,Lieu,Codepostaux,Ville,Région,Departement,URL,Telephone,...,Intérêt,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees,Musée,latitude,longitude
0,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
1,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
2,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
3,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
4,M0018,musée des Arts Décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Palais Rohan, construit de 1739 à 1742 par Rob...",NaN,Site patrimonial remarquable. Patrimoine mondi...,SPR4400040,NaN,2024-02-07,"48.581229, 7.751984",Musée,48.571265,7.767760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,M1218,"atelier-musée de l'imprimerie, Le Malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,NaN,NaN,NaN,NaN,NaN,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7381,M1218,"atelier-musée de l'imprimerie, Le Malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,NaN,NaN,NaN,NaN,NaN,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.253272,2.418897
7382,M1218,"atelier-musée de l'imprimerie, Le Malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,NaN,NaN,NaN,NaN,NaN,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7383,M1218,"atelier-musée de l'imprimerie, Le Malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,NaN,NaN,NaN,NaN,NaN,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945


In [108]:
""" concat_musee_dataframe = pd.concat([df_musee, df_geopoints], axis=1) 
concat_musee_dataframe.head(5) """

' concat_musee_dataframe = pd.concat([df_musee, df_geopoints], axis=1) \nconcat_musee_dataframe.head(5) '

In [109]:
concat_musee_dataframe.isnull().sum()


Identifiant               0
Nom_officiel              0
Adresse                 166
Lieu                   4748
Codepostaux               0
Ville                     0
Région                    0
Departement               0
URL                     741
Telephone               115
Categorie              5288
Domaine_thematique      221
Histoire                419
Atout                   398
Thèmes                  793
Artiste                4402
Personnage_phare       4183
Intérêt                1724
Protection_batiment    4446
Protection_espace      3014
Refmer                 3006
Annee_creation         6461
Date_de_mise_a_jour       0
Coordonnees               0
Musée                     0
latitude                  0
longitude                 0
dtype: int64

In [110]:
concat_musee_dataframe.head(5)

,Identifiant,Nom_officiel,Adresse,Lieu,Codepostaux,Ville,Région,Departement,URL,Telephone,...,Intérêt,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees,Musée,latitude,longitude
0,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
1,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
2,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
3,M0002,musée de La Poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,NaN,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
4,M0018,musée des Arts Décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Palais Rohan, construit de 1739 à 1742 par Rob...",NaN,Site patrimonial remarquable. Patrimoine mondi...,SPR4400040,NaN,2024-02-07,"48.581229, 7.751984",Musée,48.571265,7.767760


In [111]:
final_musee_dataframe = concat_musee_dataframe.to_csv('concat_musee_dataframe.csv', index=False)




## 4/ CONCAT MUSEE DATAFRAME - drop NAN for dataframe concatenee musee et geopoint

In [112]:
concat_musee_dataframe.isna().sum()

Identifiant               0
Nom_officiel              0
Adresse                 166
Lieu                   4748
Codepostaux               0
Ville                     0
Région                    0
Departement               0
URL                     741
Telephone               115
Categorie              5288
Domaine_thematique      221
Histoire                419
Atout                   398
Thèmes                  793
Artiste                4402
Personnage_phare       4183
Intérêt                1724
Protection_batiment    4446
Protection_espace      3014
Refmer                 3006
Annee_creation         6461
Date_de_mise_a_jour       0
Coordonnees               0
Musée                     0
latitude                  0
longitude                 0
dtype: int64

4/ MUSEE  - drop NAN

In [113]:
concat_musee_dataframe.isnull().sum()

Identifiant               0
Nom_officiel              0
Adresse                 166
Lieu                   4748
Codepostaux               0
Ville                     0
Région                    0
Departement               0
URL                     741
Telephone               115
Categorie              5288
Domaine_thematique      221
Histoire                419
Atout                   398
Thèmes                  793
Artiste                4402
Personnage_phare       4183
Intérêt                1724
Protection_batiment    4446
Protection_espace      3014
Refmer                 3006
Annee_creation         6461
Date_de_mise_a_jour       0
Coordonnees               0
Musée                     0
latitude                  0
longitude                 0
dtype: int64

In [114]:
concat_musee_dataframe = concat_musee_dataframe.fillna(0)

In [115]:
print(type(concat_musee_dataframe))

<class 'pandas.core.frame.DataFrame'>


In [116]:
concat_musee_dataframe.isna().sum()

Identifiant            0
Nom_officiel           0
Adresse                0
Lieu                   0
Codepostaux            0
Ville                  0
Région                 0
Departement            0
URL                    0
Telephone              0
Categorie              0
Domaine_thematique     0
Histoire               0
Atout                  0
Thèmes                 0
Artiste                0
Personnage_phare       0
Intérêt                0
Protection_batiment    0
Protection_espace      0
Refmer                 0
Annee_creation         0
Date_de_mise_a_jour    0
Coordonnees            0
Musée                  0
latitude               0
longitude              0
dtype: int64

In [117]:
#concat_musee_dataframe.fillna(0).astype(int)

4/ MUSEE - change word musee for Musee

In [118]:

word = 'musee' 

for i, word in enumerate(concat_musee_dataframe['Nom_officiel']):

    if isinstance(word, str) and word[0].islower():
  
        formatted_word = word[0].upper() + word[1:].lower()

        concat_musee_dataframe.at[i, 'Nom_officiel'] = formatted_word

 


In [119]:
concat_musee_dataframe

,Identifiant,Nom_officiel,Adresse,Lieu,Codepostaux,Ville,Région,Departement,URL,Telephone,...,Intérêt,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees,Musée,latitude,longitude
0,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
1,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
2,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
3,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
4,M0018,Musée des arts décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Palais Rohan, construit de 1739 à 1742 par Rob...",0,Site patrimonial remarquable. Patrimoine mondi...,SPR4400040,0,2024-02-07,"48.581229, 7.751984",Musée,48.571265,7.767760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7381,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.253272,2.418897
7382,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7383,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945


In [120]:

concat_musee_dataframe['Nom_officiel'] = concat_musee_dataframe['Nom_officiel'].apply(lambda word: word[0].upper() + word[1:].lower() if isinstance(word, str) and word[0].islower() else word)

In [121]:
concat_musee_dataframe.rename(columns={"Nom_officiel": "Nom"}, inplace=True)

In [122]:
concat_musee_dataframe

,Identifiant,Nom,Adresse,Lieu,Codepostaux,Ville,Région,Departement,URL,Telephone,...,Intérêt,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees,Musée,latitude,longitude
0,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
1,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
2,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
3,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
4,M0018,Musée des arts décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Palais Rohan, construit de 1739 à 1742 par Rob...",0,Site patrimonial remarquable. Patrimoine mondi...,SPR4400040,0,2024-02-07,"48.581229, 7.751984",Musée,48.571265,7.767760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7381,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.253272,2.418897
7382,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7383,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945


In [123]:
concat_musee_dataframe['Nom'].head(5) 


0          Musée de la poterie
1          Musée de la poterie
2          Musée de la poterie
3          Musée de la poterie
4    Musée des arts décoratifs
Name: Nom, dtype: object

In [124]:
concat_musee_dataframe.columns


Index(['Identifiant', 'Nom', 'Adresse', 'Lieu', 'Codepostaux', 'Ville',
       'Région', 'Departement', 'URL', 'Telephone', 'Categorie',
       'Domaine_thematique', 'Histoire', 'Atout', 'Thèmes', 'Artiste',
       'Personnage_phare', 'Intérêt', 'Protection_batiment',
       'Protection_espace', 'Refmer', 'Annee_creation', 'Date_de_mise_a_jour',
       'Coordonnees', 'Musée', 'latitude', 'longitude'],
      dtype='object')

In [125]:
concat_musee_dataframe

,Identifiant,Nom,Adresse,Lieu,Codepostaux,Ville,Région,Departement,URL,Telephone,...,Intérêt,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees,Musée,latitude,longitude
0,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
1,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
2,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
3,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Maison à pans de bois du début du XVIIIe siècle.,0,0,0,0,2023-04-12,"48.900348, 7.914409",Musée,48.893268,7.924354
4,M0018,Musée des arts décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Palais Rohan, construit de 1739 à 1742 par Rob...",0,Site patrimonial remarquable. Patrimoine mondi...,SPR4400040,0,2024-02-07,"48.581229, 7.751984",Musée,48.571265,7.767760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7381,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.253272,2.418897
7382,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945
7383,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,02 38 33 22 67,...,0,0,0,0,0,2023-04-19,"48.299076917985694, 2.3864524975252785",Musée,48.277597,2.351945


In [126]:
columns = ['Identifiant', 'Nom', 'Adresse', 'Lieu', 'Codepostaux',
       'Ville', 'Région', 'Departement', 'URL', 'Telephone', 'Categorie',
       'Domaine_thematique', 'Histoire', 'Atout', 'Thèmes', 'Artiste',
       'Personnage_phare', 'Intérêt', 'Annee_creation', 
       'Coordonnees', 'Musée', 'Codepostaux', 'latitude', 'longitude']

In [127]:
concat_musee_dataframe = concat_musee_dataframe[columns]
concat_musee_dataframe.head(5)

,Identifiant,Nom,Adresse,Lieu,Codepostaux,Ville,Région,Departement,URL,Telephone,...,Thèmes,Artiste,Personnage_phare,Intérêt,Annee_creation,Coordonnees,Musée,Codepostaux,latitude,longitude
0,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Arts décoratifs : Céramique,0,0,Maison à pans de bois du début du XVIIIe siècle.,0,"48.900348, 7.914409",Musée,67660,48.893268,7.924354
1,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Arts décoratifs : Céramique,0,0,Maison à pans de bois du début du XVIIIe siècle.,0,"48.900348, 7.914409",Musée,67660,48.893268,7.924354
2,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Arts décoratifs : Céramique,0,0,Maison à pans de bois du début du XVIIIe siècle.,0,"48.900348, 7.914409",Musée,67660,48.893268,7.924354
3,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Arts décoratifs : Céramique,0,0,Maison à pans de bois du début du XVIIIe siècle.,0,"48.900348, 7.914409",Musée,67660,48.893268,7.924354
4,M0018,Musée des arts décoratifs,2 place du château,palais Rohan,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,03 88 52 50 08,...,"Arts décoratifs : Céramique, Mobilier, Horloge...","Robert de Cotte, Paul et Joseph Hannong (faïen...",Cardinaux Rohan,"Palais Rohan, construit de 1739 à 1742 par Rob...",0,"48.581229, 7.751984",Musée,67000,48.571265,7.767760


In [128]:
concat_musee_dataframe[columns]

,Identifiant,Nom,Adresse,Lieu,Codepostaux,Codepostaux,Ville,Région,Departement,URL,...,Artiste,Personnage_phare,Intérêt,Annee_creation,Coordonnees,Musée,Codepostaux,Codepostaux,latitude,longitude
0,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,...,0,0,Maison à pans de bois du début du XVIIIe siècle.,0,"48.900348, 7.914409",Musée,67660,67660,48.893268,7.924354
1,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,...,0,0,Maison à pans de bois du début du XVIIIe siècle.,0,"48.900348, 7.914409",Musée,67660,67660,48.893268,7.924354
2,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,...,0,0,Maison à pans de bois du début du XVIIIe siècle.,0,"48.900348, 7.914409",Musée,67660,67660,48.893268,7.924354
3,M0002,Musée de la poterie,2 rue de Kuhlendorf,0,67660,67660,Betschdorf,Grand Est,Bas-Rhin,www.betschdorf.com/vie-culturelle-et-associati...,...,0,0,Maison à pans de bois du début du XVIIIe siècle.,0,"48.900348, 7.914409",Musée,67660,67660,48.893268,7.924354
4,M0018,Musée des arts décoratifs,2 place du château,palais Rohan,67000,67000,Strasbourg,Grand Est,Bas-Rhin,www.musees.strasbourg.eu/musee-des-arts-decora...,...,"Robert de Cotte, Paul et Joseph Hannong (faïen...",Cardinaux Rohan,"Palais Rohan, construit de 1739 à 1742 par Rob...",0,"48.581229, 7.751984",Musée,67000,67000,48.571265,7.767760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7380,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,...,0,0,0,0,"48.299076917985694, 2.3864524975252785",Musée,45330,45330,48.277597,2.351945
7381,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,...,0,0,0,0,"48.299076917985694, 2.3864524975252785",Musée,45330,45330,48.253272,2.418897
7382,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,...,0,0,0,0,"48.299076917985694, 2.3864524975252785",Musée,45330,45330,48.277597,2.351945
7383,M1218,"Atelier-musée de l'imprimerie, le malesherbois",70 avenue du Général Patton,Le Malesherbois,45330,45330,Malesherbes,Centre-Val de Loire,Loiret,a-mi.fr/,...,0,0,0,0,"48.299076917985694, 2.3864524975252785",Musée,45330,45330,48.277597,2.351945


## 5/ NEW ARCHITECTURE DATAFRAME 

In [129]:
df_site_architecture = pd.read_csv('./Data/liste-des-edifices-labellises-architecture-contemporaine-remarquable-acr.csv', sep =";")


In [143]:
df_site_architecture .head(5)

,Région,Numero_departement,Commune,INSEE,Adresse_normalisee,Coordonnees,Titre_courant,Département,Description_historique,Auteur_de_l_edifice,Description_de_l_edifice,Site architectural
0,Auvergne-Rhône-Alpes,26.0,Donzère;La Garde-Adhémart,26116,Route départementale 358 ; Pierrelatte (route de),"44.44424, 4.71027",pont à hauban dit pont Albert-Caquot,Drôme,Le pont a été construit par Albert Caquot sur ...,Caquot Albert (architecte),Pont d'une seule voie constitué par deux tabli...,Site architectural
1,Auvergne-Rhône-Alpes,26.0,Chapelle-en-Vercors (La),26074,NaN,"44.974803, 5.40436",ferme dite Les Aubaneaux,Drôme,"Suite aux bombardements, pillage et incendie d...",Descottes-Genon Marcel (architecte);Sabatou Je...,La ferme des Aubaneaux a été reconstruite en a...,Site architectural
2,Auvergne-Rhône-Alpes,74.0,Annecy,74010,"Lycée (boulevard du) 3 bis, 5, 7 ; Francs-Tire...","45.9058277995876, 6.124396526131411","ensemble de trois immeubles, dit Résidence La ...",Haute-Savoie,Ensemble de trois immeubles de logements dits ...,Cottard Robert (architecte);Charlier Robert (m...,Il s’agit d’un ensemble de 3 immeubles formant...,Site architectural
3,Auvergne-Rhône-Alpes,74.0,Cluses,74081,Sardagne (avenue de ),NaN,cité de La Sardagne,Haute-Savoie,NaN,Novarina Maurice (architecte);Plottier Richard...,NaN,Site architectural
4,Auvergne-Rhône-Alpes,26.0,Valence,26362,Victor Hugo (avenue) 378,"44.913098, 4.884487",station-service Relais du Sud,Drôme,La station service « le Relais du Sud » a été ...,Garin Henri,"Le bâtiment principal de la station service, c...",Site architectural


5/ ARCHITECTURE - Add new column Site Architectural

In [144]:
df_site_architecture['Site architectural'] = df_site_architecture.apply(lambda _: 'Site architectural', axis=1)

In [145]:
df_site_architecture['Titre_courant'].head(30)


0                  pont à hauban dit pont Albert-Caquot
1                              ferme dite Les Aubaneaux
2     ensemble de trois immeubles, dit Résidence La ...
3                                   cité de La Sardagne
4                         station-service Relais du Sud
5                          Église Saint-Joseph-des-Fins
6     immeubles de la zone d'urbanisation prioritair...
7     gare inférieure du téléphérique des Grandes-Pl...
8                   "maison dite ""Maison à Pélo Ben"""
9                  Bâtiment thermal Jules-Louis Chardon
10                        Église dite centre œcuménique
11    "zone d'aménagement concerté dite "logements d...
12                                         bibliothèque
13            École nationale supérieure d'architecture
14                               barrage dit Le Chambon
15           foyer de travailleurs dit foyer Clairvivre
16                               Cité H.L.M. Beaulieu I
17              immeuble de bureaux de la Sociét

In [146]:
print(df_site_architecture.columns)

Index(['Région', 'Numero_departement', 'Commune', 'INSEE',
       'Adresse_normalisee', 'Coordonnees', 'Titre_courant', 'Département',
       'Description_historique', 'Auteur_de_l_edifice',
       'Description_de_l_edifice', 'Site architectural'],
      dtype='object')


In [140]:
df_site_architecture = df_site_architecture.drop(['Reference_de_la_notice', 'Ancienne_reference_de_la_notice_RENV',
       'Cadre_de_l_etude', 'Ancien_nom_commune', 
       'Adresse_forme_editoriale', 'References_cadastrales','Vocable___pour_les_edifices_cultuels',
       'Dénominations', 'Destination_actuelle_de_l_edifice','Date_de_Label', 'Precisions_sur_l_interet',
       'Statut_juridique_du_proprietaire',
       'Etablissement_affectataire_de_l_edifice',
       'Auteur_de_la_photographie_autp', 
       'Materiaux_du_gros_oeuvre', 'Type_de_couverture',
       'Partie_d_elevation_exterieure', 'Materiaux_de_la_couverture',
       'Typologie_de_plan', 'Technique_du_decor_porte_de_l_edifice',
       'Indexation_iconographique_normalisee',
       'Description_de_l_elevation_interieure',
       'Emplacement_forme_structure_escalier', 'etat_de_conservation',
       'Commune_forme_editoriale', 'Elements_remarquables_dans_l_edifice', 
       'Lieudit', 'Siecle_de_la_campagne_principale_de_construction', 'Siecle_de_campagne_secondaire_de_construction', 'Datation_de_l_edifice',  'Liens_externes', 'Acces_memoire_WEB', 'Observations'], axis=1)



KeyError: "['Reference_de_la_notice', 'Ancienne_reference_de_la_notice_RENV', 'Cadre_de_l_etude', 'Ancien_nom_commune', 'Adresse_forme_editoriale', 'References_cadastrales', 'Vocable___pour_les_edifices_cultuels', 'Dénominations', 'Destination_actuelle_de_l_edifice', 'Date_de_Label', 'Precisions_sur_l_interet', 'Statut_juridique_du_proprietaire', 'Etablissement_affectataire_de_l_edifice', 'Auteur_de_la_photographie_autp', 'Materiaux_du_gros_oeuvre', 'Type_de_couverture', 'Partie_d_elevation_exterieure', 'Materiaux_de_la_couverture', 'Typologie_de_plan', 'Technique_du_decor_porte_de_l_edifice', 'Indexation_iconographique_normalisee', 'Description_de_l_elevation_interieure', 'Emplacement_forme_structure_escalier', 'etat_de_conservation', 'Commune_forme_editoriale', 'Elements_remarquables_dans_l_edifice', 'Lieudit', 'Siecle_de_la_campagne_principale_de_construction', 'Siecle_de_campagne_secondaire_de_construction', 'Datation_de_l_edifice', 'Liens_externes', 'Acces_memoire_WEB', 'Observati

In [138]:
df_site_architecture.rename

df_site_architecture.rename(columns={'Adresse_normalisee':'Adress', Titre_courant': 'Nomination', 'Description_historique': 'Description historique', 'Auteur_de_l_edifice': 'Auteur', "Description_de_l_edifice":"Description"}, inplace=True)

SyntaxError: unterminated string literal (detected at line 3) (2112517817.py, line 3)

In [157]:

df_site_architecture.rename(columns={"Adresse_normalisee": "Adresse", "Titre_courant": "Nomination",'Description_de_l_edifice':'Description', '	Description_historique': 'Description', 'Site architectural':'Site', 'Auteur_de_l_edifice':'Auteur', 'Numero_departement': 'Department' }, inplace=True)
df_site_architecture.head(5)

,Région,Department,Commune,INSEE,Adresse,Coordonnees,Nomination,Département,Description_historique,Auteur,Description,Site,latitude,longitude
0,Auvergne-Rhône-Alpes,26.0,Donzère;La Garde-Adhémart,26116,Route départementale 358 ; Pierrelatte (route de),"44.44424, 4.71027",pont à hauban dit pont Albert-Caquot,Drôme,Le pont a été construit par Albert Caquot sur ...,Caquot Albert (architecte),Pont d'une seule voie constitué par deux tabli...,Site architectural,44.44424,4.71027
1,Auvergne-Rhône-Alpes,26.0,Chapelle-en-Vercors (La),26074,NaN,"44.974803, 5.40436",ferme dite Les Aubaneaux,Drôme,"Suite aux bombardements, pillage et incendie d...",Descottes-Genon Marcel (architecte);Sabatou Je...,La ferme des Aubaneaux a été reconstruite en a...,Site architectural,44.974803,5.40436
2,Auvergne-Rhône-Alpes,74.0,Annecy,74010,"Lycée (boulevard du) 3 bis, 5, 7 ; Francs-Tire...","45.9058277995876, 6.124396526131411","ensemble de trois immeubles, dit Résidence La ...",Haute-Savoie,Ensemble de trois immeubles de logements dits ...,Cottard Robert (architecte);Charlier Robert (m...,Il s’agit d’un ensemble de 3 immeubles formant...,Site architectural,45.9058277995876,6.124396526131411
3,Auvergne-Rhône-Alpes,74.0,Cluses,74081,Sardagne (avenue de ),,cité de La Sardagne,Haute-Savoie,NaN,Novarina Maurice (architecte);Plottier Richard...,NaN,Site architectural,,None
4,Auvergne-Rhône-Alpes,26.0,Valence,26362,Victor Hugo (avenue) 378,"44.913098, 4.884487",station-service Relais du Sud,Drôme,La station service « le Relais du Sud » a été ...,Garin Henri,"Le bâtiment principal de la station service, c...",Site architectural,44.913098,4.884487


5/ ARCHITECTURE - Split Coordonées for Architecture

In [148]:
df_site_architecture['Coordonnees'] = df_site_architecture['Coordonnees'].fillna('')


In [158]:
df_site_architecture.head(5)

,Région,Department,Commune,INSEE,Adresse,Coordonnees,Nomination,Département,Description_historique,Auteur,Description,Site,latitude,longitude
0,Auvergne-Rhône-Alpes,26.0,Donzère;La Garde-Adhémart,26116,Route départementale 358 ; Pierrelatte (route de),"44.44424, 4.71027",pont à hauban dit pont Albert-Caquot,Drôme,Le pont a été construit par Albert Caquot sur ...,Caquot Albert (architecte),Pont d'une seule voie constitué par deux tabli...,Site architectural,44.44424,4.71027
1,Auvergne-Rhône-Alpes,26.0,Chapelle-en-Vercors (La),26074,NaN,"44.974803, 5.40436",ferme dite Les Aubaneaux,Drôme,"Suite aux bombardements, pillage et incendie d...",Descottes-Genon Marcel (architecte);Sabatou Je...,La ferme des Aubaneaux a été reconstruite en a...,Site architectural,44.974803,5.40436
2,Auvergne-Rhône-Alpes,74.0,Annecy,74010,"Lycée (boulevard du) 3 bis, 5, 7 ; Francs-Tire...","45.9058277995876, 6.124396526131411","ensemble de trois immeubles, dit Résidence La ...",Haute-Savoie,Ensemble de trois immeubles de logements dits ...,Cottard Robert (architecte);Charlier Robert (m...,Il s’agit d’un ensemble de 3 immeubles formant...,Site architectural,45.9058277995876,6.124396526131411
3,Auvergne-Rhône-Alpes,74.0,Cluses,74081,Sardagne (avenue de ),,cité de La Sardagne,Haute-Savoie,NaN,Novarina Maurice (architecte);Plottier Richard...,NaN,Site architectural,,None
4,Auvergne-Rhône-Alpes,26.0,Valence,26362,Victor Hugo (avenue) 378,"44.913098, 4.884487",station-service Relais du Sud,Drôme,La station service « le Relais du Sud » a été ...,Garin Henri,"Le bâtiment principal de la station service, c...",Site architectural,44.913098,4.884487


In [159]:
df_site_architecture[['latitude', 'longitude']] = df_site_architecture['Coordonnees'].str.split(', ', expand=True)
df_site_architecture.head(5)





,Région,Department,Commune,INSEE,Adresse,Coordonnees,Nomination,Département,Description_historique,Auteur,Description,Site,latitude,longitude
0,Auvergne-Rhône-Alpes,26.0,Donzère;La Garde-Adhémart,26116,Route départementale 358 ; Pierrelatte (route de),"44.44424, 4.71027",pont à hauban dit pont Albert-Caquot,Drôme,Le pont a été construit par Albert Caquot sur ...,Caquot Albert (architecte),Pont d'une seule voie constitué par deux tabli...,Site architectural,44.44424,4.71027
1,Auvergne-Rhône-Alpes,26.0,Chapelle-en-Vercors (La),26074,NaN,"44.974803, 5.40436",ferme dite Les Aubaneaux,Drôme,"Suite aux bombardements, pillage et incendie d...",Descottes-Genon Marcel (architecte);Sabatou Je...,La ferme des Aubaneaux a été reconstruite en a...,Site architectural,44.974803,5.40436
2,Auvergne-Rhône-Alpes,74.0,Annecy,74010,"Lycée (boulevard du) 3 bis, 5, 7 ; Francs-Tire...","45.9058277995876, 6.124396526131411","ensemble de trois immeubles, dit Résidence La ...",Haute-Savoie,Ensemble de trois immeubles de logements dits ...,Cottard Robert (architecte);Charlier Robert (m...,Il s’agit d’un ensemble de 3 immeubles formant...,Site architectural,45.9058277995876,6.124396526131411
3,Auvergne-Rhône-Alpes,74.0,Cluses,74081,Sardagne (avenue de ),,cité de La Sardagne,Haute-Savoie,NaN,Novarina Maurice (architecte);Plottier Richard...,NaN,Site architectural,,None
4,Auvergne-Rhône-Alpes,26.0,Valence,26362,Victor Hugo (avenue) 378,"44.913098, 4.884487",station-service Relais du Sud,Drôme,La station service « le Relais du Sud » a été ...,Garin Henri,"Le bâtiment principal de la station service, c...",Site architectural,44.913098,4.884487


In [ ]:
df_site_architecture.to_csv('architecture.csv', index=False)

## Création carte interactive avec Plotly

In [ ]:
import mplcursors
import plotly.graph_objects as go


In [165]:
fig = px.scatter_mapbox(df_site_architecture.iloc[0:1000],
                        lat=df_site_architecture['latitude'].iloc[0:1000], 
                        lon=df_site_architecture['longitude'].iloc[0:1000],
                        hover_name=df_site_architecture['Nomination'].iloc[0:1000],
                        hover_data={'latitude': False, 'longitude': False, 'Auteur': True},
                        mapbox_style="carto-positron", #fond de carte
                        zoom=6, center = {"lat": 46, "lon": 4}, #positionnement par défaut de la carte
                        custom_data=['Description' ])




fig.add_trace(go.Scattermapbox(
        lat=concat_musee_dataframe["latitude"], 
        lon=concat_musee_dataframe["longitude"],
        text=concat_musee_dataframe["Nom"],
        mode="markers",
        marker=go.scattermapbox.Marker(size=8, color='red'),  # Style des points
        hoverinfo="text"))

                        
fig.show()